In [2]:
import import_ipynb
from Functions import *
from skimage.metrics import structural_similarity as ssim
from scipy.stats import entropy

## Display Frames in List

In [2]:
def ThroughFrames(frames):
    i = 0
    while True:
        cv2.imshow('Frame', frames[i])
         # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key
            i = i - 1
            if i<0:
                i = 0
        if key == 2555904:  # Right arrow key
            i = i + 1
            if i>(len(frames)-1):
                i = len(frames)-1
    # Release the video capture object
    cv2.destroyAllWindows()

##  Entropy

In [3]:
@numba.jit
def Entropy(image):
    # Convert the image to grayscale using manual conversion
    gray_image = 0.299 * image[:, :, 2] + 0.587 * image[:, :, 1] + 0.114 * image[:, :, 0]
    # Calculate the histogram of the grayscale image
    hist, _ = np.histogram(gray_image, bins=256, range=(0, 256), density=True)
    # Avoid using SciPy's entropy function, instead calculate it manually
    hist = hist[hist > 0]  # Remove zero entries to avoid log(0)
    hist_entropy = -np.sum(hist * np.log2(hist))
    return hist_entropy

## Temporal Signal to Noise Ratio for Inconsistency

In [4]:
@numba.jit
def TSNR(image1, image2, image3):  
    if image1 is None or image2 is None:
        raise ValueError("One or both images are invalid")   
    # Calculate the absolute difference manually
    frame_diff = np.abs(image1.astype(np.float64) - image2.astype(np.float64))
    # Calculate the mean and standard deviation of the difference
    mean_diff = np.mean(frame_diff)
    std_diff = np.std(frame_diff)
    # Compute TSNR
    tsnr = mean_diff / (std_diff + 1e-10)
    return abs(1 - tsnr)

## Absolute Difference

In [5]:
@numba.jit
def Abs_Dif(image1, image2, image3):
    return np.mean(np.abs(image1 - image2))

## Optical Flow End Point Error

In [6]:
@numba.jit
def OF_EPE(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both images are invalid")
    # Manual grayscale conversion (approximating cv2.cvtColor)
    gray1 = 0.299 * image1[:, :, 2] + 0.587 * image1[:, :, 1] + 0.114 * image1[:, :, 0]
    gray2 = 0.299 * image2[:, :, 2] + 0.587 * image2[:, :, 1] + 0.114 * image2[:, :, 0]
    # Placeholder for optical flow (since Farneback can't be done with NumPy)
    # For real use, replace this with actual optical flow computation outside of Numba.
    # Example: Use np.gradient or any other available method that approximates movement.
    flow_x = np.gradient(gray2, axis=1)  # Approximation: gradient as optical flow
    flow_y = np.gradient(gray2, axis=0)
    flow = np.stack((flow_x, flow_y), axis=-1)
    # EPE (End Point Error) calculation
    epe = np.linalg.norm(flow, axis=2).mean()
    return epe

## Oprical Flow Angular Error

In [7]:
@numba.jit
def OF_AE(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both images are invalid")
    # Manual grayscale conversion (approximation of cv2.cvtColor)
    gray1 = 0.299 * image1[:, :, 2] + 0.587 * image1[:, :, 1] + 0.114 * image1[:, :, 0]
    gray2 = 0.299 * image2[:, :, 2] + 0.587 * image2[:, :, 1] + 0.114 * image2[:, :, 0] 
    # Placeholder for optical flow (since Farneback can't be done with NumPy)
    # Example: Gradient approximation for optical flow
    flow_x = np.gradient(gray2, axis=1)  # Approximation: gradient as optical flow
    flow_y = np.gradient(gray2, axis=0)
    u, v = flow_x, flow_y   
    # Magnitude and angle calculation (approximation of cv2.cartToPolar)
    magnitude = np.sqrt(u**2 + v**2)
    angle = np.arctan2(v, u)  
    # AE (Angular Error) calculation
    ae = np.mean(np.abs(angle))  # Taking the mean absolute angle error  
    return ae

## Optical Flow Error

In [ ]:
@numba.jit
def OF(img1, img2, img3):
    # Check if images are the same size
    if img1.shape != img2.shape or img2.shape != img3.shape:
        raise ValueError("All images must be of the same size")
    # Convert to grayscale using a simple weighted sum
    edges1 = 0.299 * img1[:, :, 2] + 0.587 * img1[:, :, 1] + 0.114 * img1[:, :, 0]
    edges2 = 0.299 * img2[:, :, 2] + 0.587 * img2[:, :, 1] + 0.114 * img2[:, :, 0]
    edges3 = 0.299 * img3[:, :, 2] + 0.587 * img3[:, :, 1] + 0.114 * img3[:, :, 0]
    # Approximate optical flow by using gradients (placeholder)
    flow1_x = np.gradient(edges2, axis=1) - np.gradient(edges1, axis=1)
    flow1_y = np.gradient(edges2, axis=0) - np.gradient(edges1, axis=0)
    flow2_x = np.gradient(edges3, axis=1) - np.gradient(edges2, axis=1)
    flow2_y = np.gradient(edges3, axis=0) - np.gradient(edges2, axis=0)
    # Compute the magnitude and angle of the optical flow vectors for both flows
    magnitude1 = np.sqrt(flow1_x**2 + flow1_y**2)
    angle1 = np.arctan2(flow1_y, flow1_x)
    magnitude2 = np.sqrt(flow2_x**2 + flow2_y**2)
    angle2 = np.arctan2(flow2_y, flow2_x)
    # Compute the difference between the two optical flows
    magnitude_diff = np.abs(magnitude1 - magnitude2)
    angle_diff = np.abs(angle1 - angle2)
    # Normalize differences
    magnitude_index = np.sum(magnitude_diff) / (magnitude1.size + 1e-6)  # Avoid division by zero
    angle_index = np.sum(angle_diff) / (angle1.size + 1e-6)  # Avoid division by zero
    # Combine magnitude and angle differences
    difference_index = magnitude_index + angle_index
    return difference_index

## Optical Flow Border Inconsistency Index

In [ ]:
@numba.jit
def OF_B(img1, img2, img3):
    # Check if images are the same size
    if img1.shape != img2.shape or img2.shape != img3.shape:
        raise ValueError("All images must be of the same size")
    # Simple edge detection approximation using gradients
    def edge_detect(image):
        dx = np.abs(np.gradient(image, axis=1))
        dy = np.abs(np.gradient(image, axis=0))
        return np.sqrt(dx**2 + dy**2)  
    # Apply the edge detection approximation
    edges1 = edge_detect(0.299 * img1[:, :, 2] + 0.587 * img1[:, :, 1] + 0.114 * img1[:, :, 0])
    edges2 = edge_detect(0.299 * img2[:, :, 2] + 0.587 * img2[:, :, 1] + 0.114 * img2[:, :, 0])
    edges3 = edge_detect(0.299 * img3[:, :, 2] + 0.587 * img3[:, :, 1] + 0.114 * img3[:, :, 0])
    # Approximate optical flow by using gradients (placeholder)
    flow1_x = np.gradient(edges2, axis=1) - np.gradient(edges1, axis=1)
    flow1_y = np.gradient(edges2, axis=0) - np.gradient(edges1, axis=0)
    flow2_x = np.gradient(edges3, axis=1) - np.gradient(edges2, axis=1)
    flow2_y = np.gradient(edges3, axis=0) - np.gradient(edges2, axis=0)
    # Compute the magnitude and angle of the optical flow vectors for both flows
    magnitude1 = np.sqrt(flow1_x**2 + flow1_y**2)
    angle1 = np.arctan2(flow1_y, flow1_x)
    magnitude2 = np.sqrt(flow2_x**2 + flow2_y**2)
    angle2 = np.arctan2(flow2_y, flow2_x)
    # Compute the difference between the two optical flows
    magnitude_diff = np.abs(magnitude1 - magnitude2)
    angle_diff = np.abs(angle1 - angle2)
    # Normalize differences
    magnitude_index = np.sum(magnitude_diff) / (magnitude1.size + 1e-6)  # Avoid division by zero
    angle_index = np.sum(angle_diff) / (angle1.size + 1e-6)  # Avoid division by zero
    # Combine magnitude and angle differences
    difference_index = magnitude_index + angle_index
    return difference_index

## Gray Scale Absolute Difference

In [8]:
@numba.jit
def Gray_Dif(image1, image2, image3):
    def rgb_to_gray(image):
        return (image[:, :, 2] + image[:, :, 1] + image[:, :, 0])/3
    gray1 = rgb_to_gray(image1)
    gray2 = rgb_to_gray(image2)
    # Calculate the absolute difference between the two grayscale images
    diff = np.abs(gray1 - gray2)
    # Return the mean of the difference
    return np.mean(diff)

## Temporal Structural Similarity Index Measure for Inconsistency

In [9]:
@numba.jit
def TSSIM(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    ssim_value = ssim(gray1, gray2,multichannel=True,win_size=3)
    return abs(1-ssim_value)

## Mean Squared Error

In [10]:
@numba.jit
def MSE(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    # Convert images to grayscale
    def rgb_to_gray(image):
        return 0.299 * image[:, :, 2] + 0.587 * image[:, :, 1] + 0.114 * image[:, :, 0]
    gray1 = rgb_to_gray(image1)
    gray2 = rgb_to_gray(image2)
    # Compute the Mean Squared Error
    mse = np.mean((gray1 - gray2) ** 2)
    return mse

## Border Error

In [11]:
@numba.jit
def Border_Err(image1, image2, image3):
    lower_threshold = 50
    upper_threshold = 255
    if len(image1.shape) > 1:
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    if len(image2.shape) > 1:
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    canny1 = cv2.Canny(image1, lower_threshold, upper_threshold)
    canny2 = cv2.Canny(image2, lower_threshold, upper_threshold)
    # Calculate the absolute difference between the two Canny images
    abs_diff = cv2.absdiff(canny1, canny2)
    return np.mean(abs_diff)

## Color Range Consistency

In [12]:
@numba.jit
def CRC(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")    
    # Convert images to float64
    image1 = image1.astype(np.float64)
    image2 = image2.astype(np.float64)  
    # Calculate differences for each channel
    def channel_diff(image1, image2, channel):
        min1 = np.min(image1[:,:,channel])
        max1 = np.max(image1[:,:,channel])
        min2 = np.min(image2[:,:,channel])
        max2 = np.max(image2[:,:,channel])
        diff_min = abs(min1 - min2)
        diff_max = abs(max1 - max2)
        return diff_min, diff_max   
    diff_r_min, diff_r_max = channel_diff(image1, image2, 2)  # Red channel
    diff_g_min, diff_g_max = channel_diff(image1, image2, 1)  # Green channel
    diff_b_min, diff_b_max = channel_diff(image1, image2, 0)  # Blue channel   
    # Calculate CRC index
    crci = (diff_r_min + diff_r_max + diff_g_min + diff_g_max + diff_b_min + diff_b_max) / 6    
    return crci

## Entropy Difference

In [13]:
@numba.numba.jit
def Entropy_Dif(image1,image2, image3):
    return abs(Entropy(image1)-Entropy(image2))

# Frequency Magnitude Difference

In [ ]:
@numba.jit
def Freq_Dif(img1, img2, img3):
    # Check if images are the same size
    if img1.shape != img2.shape:
        raise ValueError("Images must be of the same size")  
    # Compute FFT
    f1 = np.fft.fft2(img1)
    f2 = np.fft.fft2(img2)  
    # Shift zero frequency component to the center
    f1_shifted = np.fft.fftshift(f1)
    f2_shifted = np.fft.fftshift(f2)  
    # Compute magnitude spectrum
    magnitude1 = np.abs(f1_shifted)
    magnitude2 = np.abs(f2_shifted)  
    # Compute the difference in magnitude spectra
    magnitude_diff = np.abs(magnitude1 - magnitude2)
    # Compute the difference index
    sum_magnitude1 = np.sum(magnitude1)
    sum_magnitude2 = np.sum(magnitude2)
    sum_magnitude_diff = np.sum(magnitude_diff)
    difference_index = sum_magnitude_diff / (sum_magnitude1 + sum_magnitude2 + 1e-6)  # Avoid division by zero
    return difference_index

## Combined All Metrics

In [14]:
def Combined_Metrics(image1, image2,op):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    tsnr = TSNR(image1,image2)
    adif = Abs_Dif(image1,image2)
    epe = OF_EPE(image1,image2)
    ae = OF_AE(image1,image2)
    gray = Gray_Dif(image1,image2)
    ssim_value = TSSIM(image1,image2)
    mse_value = MSE(image1,image2)
    border_consistency_value = Border_Err(image1,image2)
    crci_value = CRC(image1,image2)
    ent = Entropy_Dif(image1,image2)
    # Normalize and combine the metrics into a single consistency index
    metrics = np.array([tsnr,adif, epe, ae, gray,ssim_value, mse_value, border_consistency_value, crci_value, ent])
    normalized_metrics = (metrics - np.min(metrics)) / (np.max(metrics) - np.min(metrics))
    combined_consistency_index = np.mean(normalized_metrics)
    #print(metrics)
    # Z-score normalization
    mean = np.mean(metrics)
    std = np.std(metrics)
    Z_metrics = (metrics - mean) / std
    
    if op=="norm":
        return combined_consistency_index
    if op=="mean":
        return np.mean(metrics)
    if op=="log":
        metrics[metrics <= 0] = 1e-10
        return np.sum(np.log(metrics))
    if op=="Z":
        return np.mean(Z_metrics)

## Mixed Metrics

In [15]:
@numba.jit
def Mix_Metrics(image1, image2,image3,op="mean",M = [TSSIM,MSE],W=[1,1]):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    metrics = []
    for i in range(len(M)):
        #print(image1.shape,image2.shape)
        metrics.append(W[i]*M[i](image1,image2,image3))
    # Normalize and combine the metrics into a single consistency index
    metrics = np.array(metrics)
    normalized_metrics = (metrics - np.min(metrics)) / (np.max(metrics) - np.min(metrics))
    combined_consistency_index = np.mean(normalized_metrics)
    #print(metrics)
    # Z-score normalization
    mean = np.mean(metrics)
    std = np.std(metrics)
    Z_metrics = (metrics - mean) / std
    
    if op=="norm":
        return combined_consistency_index
    if op=="mean":
        return np.mean(metrics)
    if op=="log":
        metrics[metrics <= 0] = 1e-10
        return np.sum(np.log(metrics))
    if op=="Z":
        return np.mean(Z_metrics)

## WIndowed Max Inconsistency

In [16]:
def WMax_Inconsistency(img1, img2, img3, wsize=(3,3), step=(3,3), Func=Combined_Metrics, op="mean",M=[],Weights=[]):
    kw,kh = wsize
    H,W,_ = img1.shape
    sw,sh = step
    maxC = 0
    maxR = [0,0,kw,kh]
    # Calculate the output dimensions
    output_height = (H - kh) // sh + 1
    output_width = (W - kw) // sw + 1
    # Initialize the output array
    result = np.zeros((output_height, output_width))
    # Perform the convolution
    for i in range(0, output_height*sh, sh):
        #print('Row: ',i,'/',output_height*sh,end='\r')
        for j in range(0, output_width*sw, sw):
            # Extract the region of interest
            region1 = img1[i:i + kh, j:j + kw]
            region2 = img2[i:i + kh, j:j + kw]
            region3 = img3[i:i + kh, j:j + kw]
            # Perform element-wise multiplication and sum the result
            if Func==Combined_Metrics or Func==Mix_Metrics:
                if len(M)>0 and Func==Mix_Metrics:
                    result[i // sh, j // sw] = Func(region1, region2, region3,op,M,Weights)
                else:
                    result[i // sh, j // sw] = Func(region1, region2,region3,op)
            else:
                result[i // sh, j // sw] = Func(region1, region2,region3)
            if result[i//sh,j//sw]>=maxC:
                maxR = [i,i+kh,j,j+kw] 
                maxC = result[i//sh,j//sw]
            if (j+sw+kw)>W:
                break
        if (i+sh+kh)>H:
            break
    #print('')
    return result,maxR

## Video Consistency

In [17]:
def Vid_consistency(F,Func=Combined_Metrics,op="mean",M=[],W=[]):
    C = []
    for i in range(len(F)-2):
        if Func==Combined_Metrics or Func==Mix_Metrics:
            if len(M)>0 and Func==Mix_Metrics:
                C.append(Func(F[i],F[i+1],F[i+2],op,M,W))
            else:
                C.append(Func(F[i],F[i+1],F[i+2],op))
        else:
            C.append(Func(F[i],F[i+1],F[i+2]))
    return np.mean(np.asarray(C)),C

## Windowed Video Consistency

In [18]:
def Vid_consistency_W(F,wsize=(3,3),step=(3,3),Func=Combined_Metrics,op="mean",M=[],W=[]):
    C = []
    for i in range(len(F)-1):
        print('Frame: ',i+1,'/',len(F)-1,end='\r')
        Metrics,Region = WMax_Inconsistency(F[i],F[i+1],wsize,step,Func,op,W)
        C.append(np.mean(Metrics))
    return np.mean(np.asarray(C))

## Draw Window with Max Inconsistency

In [19]:
#@jit
def DrawInconsistancy(F,wsize=(50,50),step=(50,50),Func=Combined_Metrics,op="mean",M=[],W=[]):
    L = 3
    DI = np.copy(F)
    for i in range(len(F)-2):
        print('Frame: ',i+1,'/',len(F)-1,end='\r')
        Metrics,Region = WMax_Inconsistency(F[i],F[i+1],F[i+2],wsize,step,Func,op,M,W)
        #print((Region[0],Region[2]),(Region[1],Region[3]))
        DI[i+1][Region[0]:Region[0]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[2]:Region[2]+L] = [0,255,0] 
        DI[i+1][Region[1]:Region[1]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[3]:Region[3]+L] = [0,255,0] 
        #DI[i+1] = cv2.putText(DI[i+1],str(np.mean(Metrics)),(10,20),cv2.FONT_HERSHEY_SIMPLEX,0.25,(0,0,255),1,cv2.LINE_AA) 
    return DI

## Draw Window with Max Inconsistency of Diffrent Window Sizes

In [20]:
#@jit
def DrawInconsistancy1(F,Func=Combined_Metrics,op="mean",M=[],W=[]):
    L = 3
    DI = np.copy(F)
    for i in range(1,len(F)-1):
        print('Frame: ',i,'/',len(F)-1,end='\r')
        R = -1000
        Region = [0,0,0,0]
        for j in range(50,100,50):
            Metrics,Reg= WMax_Inconsistency(F[i-1],F[i],F[i+1],(j,j),(j//2,j//2),Func,op,M,W)
            if np.mean(Metrics)>R:
                R = np.mean(Metrics)
                Region = Reg
        #print((Region[0],Region[2]),(Region[1],Region[3]))
        
        DI[i+1][Region[0]:Region[0]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[2]:Region[2]+L] = [0,255,0] 
        DI[i+1][Region[1]:Region[1]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[3]:Region[3]+L] = [0,255,0] 
        #DI[i+1] = cv2.putText(DI[i+1],str(R),(10,20),cv2.FONT_HERSHEY_SIMPLEX,0.25,(0,0,255),1,cv2.LINE_AA) 
    return DI

In [21]:
#@jit
def InconsistentRegion(F,Func=Combined_Metrics,op="mean",M=[],W=[]):
    L = 3
    h,w,_ = F[0].shape
    DI = [np.zeros((h,w)) for _ in F]
    for i in range(len(F)-2):
        #print('Frame: ',i+1,'/',len(F)-1,end='\r')
        R = -1000
        Region = [0,0,0,0]
        for j in range(20,100,20):
            Metrics,Reg= WMax_Inconsistency(F[i],F[i+1],F[i+2],(j,j),(j//2,j//2),Func,op,M,W)
            if np.mean(Metrics)>R:
                R = np.mean(Metrics)
                Region = Reg
        #print((Region[0],Region[2]),(Region[1],Region[3]))
        
        DI[i+1][Region[0]:Region[1],Region[2]:Region[3]] = 255
    return DI,R

# Experiments

F = read_images("saved_frames")
F[3] = F[3][:,:,:3]
F[6] = F[6][:,:,:3]
F[9] = F[9][:,:,:3]
F[14] = F[14][:,:,:3]
I = DrawInconsistancy(F[:13],(50,50),(10,10),Mix_Metrics,"mean",[CRC,Border_Err,Abs_Dif],[2,1.5,10])

ThroughFrames(I)

F = read_images("saved_frames")
F[3] = F[3][:,:,:3]
F[6] = F[6][:,:,:3]
F[9] = F[9][:,:,:3]
F[14] = F[14][:,:,:3]
I = DrawInconsistancy1(F[:13],Mix_Metrics,"mean",[CRC,Border_Err],[1.5,1])

ThroughFrames(I)

In [22]:
#Check which values with min mean consistency and which max mean consistency. Ex: 1-ssim
#Weights of each metric
#Which metrics are the best?
#Look for change in entropy too

In [23]:
#Buscar origen del ruido
#Checar si el ruido proviene desde el origen
#Trabajar con fracuencias
#DNN para cada tipo de error
#Cambiar parametros de caricaturizacion por medio de DNN

#Lista de parametros e indices para identificar cuales son los mejores valores
#Algoritmos Geneticos
#For x,y in zip(X,Y) 

#Para indices de inconsistencia y para caricaturizacion
#CHecar con frames pasados (mean,var,etc) de cada parametro